<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <h2>4.1-BLEU,  ROUGE and N-Grams. </h2>
    <h3>Evaluating translations with BLEU</h3>
</div>

by [Pere Martra](https://www.linkedin.com/in/pere-martra/)
________
Models: nllb-200-distilled-600M

Colab environment: CPU.

Keys:
* Bleu Evaluation.
* Translation Pipeline.
* Google Translator API.
______



In this notebook, we will use the BLEU metric to compare the quality of two different approaches for performing translations.

As my primary language is Spanish, I will translate a few lines from the beginning of this chapter 4-Evaluating Models from my book [Large Language Models Projects](https://www.amazon.com/Pere-Martra-Manonelles/dp/B0D6XQ44ZP)from English to Spanish.

My translations will be taken as the reference translations. In other words, they will be used as the basis upon which the quality of the automatic translations will be determined.



In [1]:
!pip install -q googletrans==3.1.0a0
!pip install -q evaluate==0.4.2
!pip install -q transformers==4.42.4

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.2.3 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
openai 1.57.4 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 k

In [2]:
from googletrans import Translator
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import evaluate

In [3]:
#Sentences to Translate.
sentences = [
    "In the previous chapters, you've mainly seen how to work with OpenAI models, and you've had a very practical introduction to Hugging Face's open-source models, the use of embeddings, vector databases, and agents.",
    "These have been very practical chapters in which I've tried to gradually introduce concepts that have allowed you, or at least I hope so, to scale up your knowledge and start creating projects using the current technology stack of large language models."
    ]

In [4]:
#Spanish Translation References.
reference_translations = [
    ["En los capítulos anteriores has visto mayoritariamente como trabajar con los modelos de OpenAI, y has tenido una introducción muy práctica a los modelos Open Source de Hugging Face, al uso de embeddings, las bases de datos vectoriales, los agentes."],
    ["Han sido capítulos muy prácticos en los que he intentado ir introduciendo conceptos que te han permitido, o eso espero, ir escalando en tus conocimientos y empezar a crear proyectos usando el stack tecnológico actual de los grandes modelos de lenguaje."]
    ]

We will perform the first translation using the NLLB model, a small model specialized in performing translations, which we will retrieve from Hugging Face.

In [5]:
model_id = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

When creating the pipeline, we pass the source language and the target language of the translation to it.

In [6]:
translator = pipeline('translation', model=model, tokenizer=tokenizer,
                        src_lang="eng_Latn", tgt_lang="spa_Latn")

When calling a pipeline, you must specify which task or action you want to perform
with the model. In this case, I indicated that I wanted to perform the “translation” task.
However, I could have been more specific and indicated that I wanted to perform the
“translation_en_to_es” task, in which case I wouldn’t have needed to pass the src_lang
and tgt_lang parameters. But it’s not that simple; I don’t know how the transformers
library is implemented, and using one pipeline or another doesn’t always yield the same
results

In [7]:
translations_nllb = []

for text in sentences:
  print ("to translate: " + text)
  translation = ""
  translation = translator(text)

  #Add the summary to summaries list
  translations_nllb += translation[0].values()

to translate: In the previous chapters, you've mainly seen how to work with OpenAI models, and you've had a very practical introduction to Hugging Face's open-source models, the use of embeddings, vector databases, and agents.
to translate: These have been very practical chapters in which I've tried to gradually introduce concepts that have allowed you, or at least I hope so, to scale up your knowledge and start creating projects using the current technology stack of large language models.


This code iterates through the list containing the texts to be
translated and translates them using the previously created pipeline. The result is stored
in a list where all generated translations are kept

Now we have the translations stored in the list 'translations_nllb'.

In [8]:
translations_nllb

['En los capítulos anteriores, han visto principalmente cómo trabajar con modelos OpenAI, y han tenido una introducción muy práctica a los modelos de código abierto de Hugging Face, el uso de embebidos, bases de datos vectoriales y agentes.',
 'Estos han sido capítulos muy prácticos en los que he intentado introducir gradualmente conceptos que han permitido, o al menos espero que lo hagan, ampliar sus conocimientos y comenzar a crear proyectos utilizando la tecnología actual de los modelos de lenguaje grande.']

##Create Translations with Google Traslator.

As a second source for translations, we will use the Google Translator API.

In [9]:
translator_google = Translator()

In [10]:
translations_google = []

for text in sentences:
  print ("to translate: " + text)
  translation = ""
  translation = translator_google.translate(text, dest="es")

  #Add the summary to summaries list
  translations_google.append(translation.text)
  print (translation.text)

to translate: In the previous chapters, you've mainly seen how to work with OpenAI models, and you've had a very practical introduction to Hugging Face's open-source models, the use of embeddings, vector databases, and agents.
En los capítulos anteriores, vio principalmente cómo trabajar con modelos OpenAI y tuvo una introducción muy práctica a los modelos de código abierto de Hugging Face, el uso de incrustaciones, bases de datos vectoriales y agentes.
to translate: These have been very practical chapters in which I've tried to gradually introduce concepts that have allowed you, or at least I hope so, to scale up your knowledge and start creating projects using the current technology stack of large language models.
Estos han sido capítulos muy prácticos en los que he intentado introducir gradualmente conceptos que te han permitido, o al menos eso espero, ampliar tus conocimientos y empezar a crear proyectos utilizando la tecnología actual de grandes modelos de lenguaje.


In this list, we have the translations created by Google.

In [11]:
translations_google

['En los capítulos anteriores, vio principalmente cómo trabajar con modelos OpenAI y tuvo una introducción muy práctica a los modelos de código abierto de Hugging Face, el uso de incrustaciones, bases de datos vectoriales y agentes.',
 'Estos han sido capítulos muy prácticos en los que he intentado introducir gradualmente conceptos que te han permitido, o al menos eso espero, ampliar tus conocimientos y empezar a crear proyectos utilizando la tecnología actual de grandes modelos de lenguaje.']

## Evaluate translations with BLEU

We will use the BLEU implementation from the Evaluate library by Hugging Face.

In [12]:
bleu = evaluate.load('bleu')

In [13]:
results_nllb = bleu.compute(predictions=translations_nllb, references=reference_translations)


To obtain the metrics, we pass the translated text and the reference text to the BLEU function.

Note that the translated text is a list of translations:
["Translation1", "Translation2"]

Whereas the reference texts are a list of lists of text. This allows for providing multiple references per translation:

[["reference1 Translation1", "reference2 Translation1"],
["reference2 Translation2", "reference2 Translation2"]]


In [14]:
results_google = bleu.compute(predictions=translations_google, references=reference_translations)

In [15]:
print(results_nllb)

{'bleu': 0.3686324165619373, 'precisions': [0.7159090909090909, 0.47674418604651164, 0.30952380952380953, 0.18292682926829268], 'brevity_penalty': 0.988700685876667, 'length_ratio': 0.9887640449438202, 'translation_length': 88, 'reference_length': 89}


In [16]:
print(results_google)

{'bleu': 0.44975901966417653, 'precisions': [0.7710843373493976, 0.5679012345679012, 0.4177215189873418, 0.2987012987012987], 'brevity_penalty': 0.9302618655343314, 'length_ratio': 0.9325842696629213, 'translation_length': 83, 'reference_length': 89}


It appears that the translation performed by the Google API is significantly better than the one performed by the NLLB model.
Explanation of the returned values:

* BLEU: A value between 0 and 1 is returned. The closer this value is to 1, the better the translation. The two values we obtained with our translations are below 0.5. This might lead us to think that they are poor-quality translations, but nothing could be further from the truth. A value between 0.3 and 0.4, though not perfect, cannot
be considered bad; it retains the meaning and is understandable. If the BLEU value is between 0.4 and 0.5, it’s considered a very good translation. Values above 0.6 are rarely seen, even in human
translators. Therefore, we could say that the translation performed by NLLB is correct, while the Google Translation could be considered very good.

* Precisions: Precision evaluates the number of identical n-grams
found between the translation being evaluated and the reference
translation. As you can see, it returns four precision values,
corresponding to 1-gram, 2-gram, 3-gram, and 4-gram. Naturally,
the precision value decreases as the n-gram size increases. In the
translations we’re evaluating, NLLB achieves a 0.72 precision for
the first data point, while Google achieves 0.77. This indicates that both translations likely use a similar set of words to the reference translation. The most significant difference lies in how the precision decreases as the n-gram size increases. For the last precision
value, NLLB obtains a 0.18, compared to Google’s 0.30. This is a considerable difference, suggesting that the sentence structure Google produces is much closer to the reference translation than the
one NLLB achieves.
* Brevity_penalty: The brevity penalty is primarily an internal multiplier that affects the overall BLEU score, penalizing translations that contain fewer words than the reference text. Values of 1 or
higher indicate that the translated text contains more words than the reference text. NLLB has a brevity penalty of 0.98, while Google has
a penalty of 0.93. This tells us two things: first, that NLLB has created a translation with a length very similar to the reference, and second,
that if Google had used a couple more words, its score would have been even better, further surpassing NLLB.
* Length_ratio: Indicates the relationship between the length of the generated text and the reference texts. The closer it is to 1, the more
similar the lengths of the texts are. It is calculated using translation_length and reference_length.
